In [ ]:
!unzip /content/Test.zip

In [1]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib

In [2]:
suffix = '.pgm'
kaggle_files = os.listdir('../Test/Test')
pgm_kaggle_files = [filename for filename in kaggle_files if filename.endswith(suffix)]

In [3]:
len(pgm_kaggle_files)

8000

In [4]:
kaggle_patches = []
for filename in tqdm(pgm_kaggle_files):
  path = '../Test/Test/' + filename
  with open(path, 'rb') as pgmf:
      image = plt.imread(pgmf)
  kaggle_patches.append(image)

100%|██████████| 8000/8000 [00:01<00:00, 4774.64it/s]


In [5]:
pgm_kaggle_files_id = []

for filename in tqdm(pgm_kaggle_files):
  pgm_kaggle_files_id.append(filename[5:-4])

100%|██████████| 8000/8000 [00:00<00:00, 2324036.02it/s]


In [6]:
# Aquí debe utilizar su modelo
model = joblib.load('../modelos/random_forest_100_315v1.pkl')

In [9]:
# Aquí debe procesar las imagenes de pgm_kaggle_files
pca_model = joblib.load('../modelos/pca_final_130_315v1.pkl')
pgm_kaggle_array = np.stack(kaggle_patches)
pgm_kaggle_array = pgm_kaggle_array / 255.0  # Normalizar las imágenes
kaggle_images_flatten = np.array([im.flatten() for im in tqdm(pgm_kaggle_array, desc='Construyendo X')])
X_pca_kag = pca_model.transform(kaggle_images_flatten)


Construyendo X: 100%|██████████| 8000/8000 [00:00<00:00, 65478.32it/s]


In [10]:
X_pca_kag.shape

(8000, 130)

In [11]:
# Predicción
y_kag = model.predict(X_pca_kag)

In [12]:
# Predicción + ID
y_kag_dic = {pgm_kaggle_files_id[i]: y_kag[i] for i in range(len(pgm_kaggle_files_id))}
kaggle_hat = pd.DataFrame(list(y_kag_dic.items()), columns=['id', 'target_feature'])
kaggle_hat['id'] = kaggle_hat['id'].astype(int)
kaggle_hat['target_feature'] = kaggle_hat['target_feature'].astype(int)
kaggle_hat.sort_values(by='id', inplace=True)

In [13]:
kaggle_hat.head()

,id,target_feature
0,0,0
1,1,0
1112,2,0
2223,3,0
3334,4,0


In [14]:
kaggle_hat.value_counts('target_feature')

target_feature
0    7606
1     394
Name: count, dtype: int64

In [15]:
# Guardamos en un .csv para subir a kaggle
submission_name = 'prueba_4'
kaggle_hat.to_csv('../submissions/'+ submission_name + '.csv', index=False)